In [2]:
import numpy as np
import tensorflow as tf
import json
import re
import gensim
from gensim.models import Word2Vec
import jieba

word2vec = Word2Vec.load('../../word2vec/zh_fb_ptt.bin')

C:\Users\guanyuan2\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
concept = []

with open('../../Story/Concept_01.txt', encoding = 'utf8') as c:
    concept = c.readlines()
    
with open('../../Story/Concept_02.txt', encoding = 'utf8') as c:
    concept += c.readlines()
    
with open('../../Story/Concept_03.txt') as c:
    concept += c.readlines()

with open('../../Story/Concept_04.txt') as c:
    concept += c.readlines()
    
story = []
    
with open('../../Story/Story_01.txt', encoding = 'utf8') as c:
    story = c.readlines()
    
with open('../../Story/Story_02.txt', encoding = 'utf8') as c:
    story += c.readlines()
    
with open('../../Story/Story_03.txt') as c:
    story += c.readlines()
    
with open('../../Story/Story_04.txt') as c:
    story += c.readlines()
    
for i in range(0, len(concept)):
    concept[i] = concept[i].split(' ')
    concept[i].pop()
    
for i in range(len(story)):
    story[i] = story[i].replace('\n', '')
    
print(len(concept), len(story))

X_train_emb = []
X_train = []

y_train_emb = []
y_train = []

comma_period = ['，', '。']

tokens = ['GO',
          'EOS',
          'unknown',
          '，',
          '。']

relations = ['CapableOf', 
             'Causes', 
             'CausesDesire', 
             'HasFirstSubevent', 
             'HasSubevent', 
             'MotivatedByGoal', 
             'UsedFor']

vocab_list = tokens + relations

ones_init = np.ones(shape = (1, 300), dtype = np.float)

relations_embedding = []
relations_id = []

tokens_id = []
tokens_embedding = []

x = 5

for i in range(0, x):
    tokens_id += [i]
    tokens_embedding += [ones_init * i / 1000]
    
for i in range(x, x + 7):
    relations_id += [i]
    relations_embedding += [ones_init * i / 100]

sigma = 1e-8

def build_vocab_list(vl, w):
    if w in vl:
        return vl, vl.index(w)
    else:
        vl += [w]
        return vl, vl.index(w)

def np_arr_concat(x, y):
    if len(x) == 0:
        return np.array(y)
    else:
        return np.concatenate((x, y), axis = 0)

for a in range(len(concept)):
    word_vector = np.array([])
    id_vector = []
    for i in range(len(concept[a])):
        w = concept[a][i]
        if w in relations:
            i = relations.index(w)
            word_vector = np_arr_concat(word_vector, relations_embedding[i])
            vocab_list, vli = build_vocab_list(vocab_list, w)
            id_vector += [vli]
            
        else:
            try:
                wvec = word2vec[w] + sigma
                word_vector = np_arr_concat(word_vector, [wvec])
                vocab_list, vli = build_vocab_list(vocab_list, w)
                id_vector += [vli]
            except:
                segs = jieba.cut(w)
                for seg in segs:
                    try:
                        wvec = word2vec[seg] + sigma
                        word_vector = np_arr_concat(word_vector, [wvec])
                        vocab_list, vli = build_vocab_list(vocab_list, seg)
                        id_vector += [vli]
                    except:
                        word_vector = np_arr_concat(word_vector, tokens_embedding[2])
                        id_vector += [tokens_id[2]]
                        
    word_vector = np_arr_concat(word_vector, tokens_embedding[1])
    id_vector += [tokens_id[1]]
    X_train_emb += [word_vector]
    X_train += [id_vector]
        
print(len(X_train_emb))
print(len(X_train))
print(len(vocab_list))

''' target needs to start with <GO>'''
for a in range(len(story)):
    word_vector = np.array(tokens_embedding[0])
    id_vector = [tokens_id[0]]
    segs = jieba.cut(story[a])
    for seg in segs:
        if seg in comma_period:
            i = comma_period.index(seg)
            word_vector = np_arr_concat(word_vector, tokens_embedding[i + 3])
            vocab_list, vli = build_vocab_list(vocab_list, seg)
            id_vector += [tokens_id[i + 3]]
        else:
            try:
                wvec = word2vec[seg] + sigma
                word_vector = np_arr_concat(word_vector, [wvec])
                vocab_list, vli = build_vocab_list(vocab_list, seg)
                id_vector += [vli]
            except:
                word_vector = np_arr_concat(word_vector, tokens_embedding[2])
                id_vector += [tokens_id[2]]
    word_vector = np_arr_concat(word_vector, tokens_embedding[1])
    id_vector += [tokens_id[1]]
    y_train_emb += [word_vector]
    y_train += [id_vector]

print(len(y_train_emb))
print(len(y_train))
print(len(vocab_list))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\GUANYU~1\AppData\Local\Temp\jieba.cache


817 817


Loading model cost 0.553 seconds.
Prefix dict has been built succesfully.


817
817
814
817
817
1239


In [9]:
embedding_matrix = np.array([])

x = 5
for i in range(0, x):
    embedding_matrix = np_arr_concat(embedding_matrix, ones_init * i / 1000) 
    
for i in range(x, x + 7):
    embedding_matrix = np_arr_concat(embedding_matrix, ones_init * i / 100) 

for i, vocab in enumerate(vocab_list):
    if i >= 12:
        embedding_matrix = np_arr_concat(embedding_matrix, [word2vec[vocab]]) 

In [283]:
a = 19
s = 0
for n in embedding_matrix[a]:
    s = s + n
print(s)
print(vocab_list[a])

48.7565164801
看


In [4]:
import math
import sys
import numpy as np
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple, GRUCell
from tensorflow.python.layers import core as layers_core
import Touka

class Seq2SeqModel():
    def __init__(self, 
                 encoder_num_units, 
                 decoder_num_units, 
                 embedding_size,
                 num_layers,
                 vocab_size, 
                 batch_size,
                 bidirectional = False,
                 attention = False,
                 pre_emb = False,
                 time_major = False,
                 beam_search = False,
                 beam_width = None):
        
        self.bidirectional = bidirectional
        self.attention = attention
        self.pre_emb = pre_emb
        self.time_major = time_major
        self.beam_search = beam_search

        self.embedding_size = embedding_size
        self.num_layers = num_layers
        self.encoder_num_units = encoder_num_units
        self.decoder_num_units = decoder_num_units

        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.beam_width = beam_width

        self._make_graph()

    def _make_graph(self):

        self._init_placeholders()
        
        self._init_decoder_train_connectors()
        
        self._init_embedding()

        if self.bidirectional:
            self._init_bidirectional_encoder()
        else:
            self._init_encoder()

        self._init_decoder()

        self._init_optimizer()

    def _init_placeholders(self):
        self.encoder_inputs = tf.placeholder(
            shape = (None, None),
            dtype = tf.int32,
            name = 'encoder_inputs'
        )

        self.encoder_inputs_length = tf.placeholder(
            shape=(None,),
            dtype = tf.int32,
            name='encoder_inputs_length',
        )
        
        self.decoder_targets = tf.placeholder(
            shape=(None, None),
            dtype=tf.int32,
            name='decoder_targets',
        )
        
        self.decoder_targets_length = tf.placeholder(
            shape=(None,),
            dtype=tf.int32,
            name='decoder_targets_length',
        )
        
        if self.pre_emb:
            self.embedding_placeholder = tf.placeholder(tf.float32, [self.vocab_size, self.embedding_size])
        
    def make_train_inputs(self, x, y, e_m = None):
        inputs_id_, inputs_id_length_ = Touka.prepare_batch(inputs = x, 
                                                            dim = 2,
                                                            time_major = self.time_major)
        targets_id_, targets_id_length_ = Touka.prepare_batch(inputs = y,
                                                              dim = 2,
                                                              time_major = self.time_major)
        if self.pre_emb:
            return {
                self.encoder_inputs: inputs_id_,
                self.encoder_inputs_length: inputs_id_length_,
                self.decoder_targets_length: targets_id_length_,
                self.decoder_targets: targets_id_,
                self.embedding_placeholder: e_m
            }
        else:
            return {
                self.encoder_inputs: inputs_id_,
                self.encoder_inputs_length: inputs_id_length_,
                self.decoder_targets_length: targets_id_length_,
                self.decoder_targets: targets_id_
            }
            

    def _init_decoder_train_connectors(self):
        with tf.name_scope('DecoderTrainFeeds'):                
            self.decoder_train_length = self.decoder_targets_length
            self.loss_weights = tf.ones([
                self.batch_size,
                tf.reduce_max(self.decoder_train_length)
            ], dtype=tf.float32)

    def _init_embedding(self):
        if self.pre_emb:
            self.embedding_encoder = tf.Variable(tf.constant(0.0, shape=[self.vocab_size, self.embedding_size]),
                                                 trainable=False)
            self.embedding_decoder = tf.Variable(tf.constant(0.0, shape=[self.vocab_size, self.embedding_size]),
                                                 trainable=False)
            enc_embedding_init = self.embedding_encoder.assign(self.embedding_placeholder)
            dec_embedding_init = self.embedding_decoder.assign(self.embedding_placeholder)

            self.encoder_embedding_inputs = tf.nn.embedding_lookup(
                self.embedding_encoder, self.encoder_inputs)
            self.decoder_embedding_inputs = tf.nn.embedding_lookup(
                self.embedding_decoder, self.decoder_targets)
            
        else:
            self.embedding_encoder = tf.Variable(tf.random_uniform([self.vocab_size, 
                                                            self.embedding_size]))
            self.encoder_embedding_inputs = tf.nn.embedding_lookup(
                self.embedding_encoder, self.encoder_inputs)

            self.embedding_decoder = tf.Variable(tf.random_uniform([self.vocab_size, 
                                                            self.embedding_size]))
            
            self.decoder_embedding_inputs = tf.nn.embedding_lookup(
                self.embedding_decoder, self.decoder_targets)

    def _init_encoder(self):
        def make_cell(rnn_size):
            enc_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
            return enc_cell
        with tf.variable_scope("Encoder") as scope:
            num_layers = self.num_layers
            encoder_cell = tf.contrib.rnn.MultiRNNCell([make_cell(self.encoder_num_units) for _ in range(num_layers)])
            self.encoder_outputs, self.encoder_state = tf.nn.dynamic_rnn(
                cell = encoder_cell, 
                inputs = self.encoder_embedding_inputs,
                sequence_length = self.encoder_inputs_length, 
                time_major = self.time_major,
                dtype = tf.float32
            )

    def _init_bidirectional_encoder(self):
        '''
            to be fixed
            
        '''
#         with tf.variable_scope("Bidirectional_Encoder") as scope:
#             def make_cell(rnn_size):
#                 enc_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
#                 return enc_cell
#             num_layers = 4
#             encoder_cell = tf.contrib.rnn.MultiRNNCell([make_cell(self.encoder_num_units) for _ in range(num_layers)])
#             bi_enc_outputs, bi_enc_state = tf.nn.bidirectional_dynamic_rnn(
#                 cell_fw = encoder_cell, 
#                 cell_bw = encoder_cell, 
#                 inputs = self.encoder_embedding_inputs,
#                 sequence_length = self.encoder_inputs_length, 
#                 time_major = True,
#                 dtype = tf.float32
#             )
#             self.encoder_outputs = tf.concat(bi_enc_outputs, 2)
            
#             encoder_state_c = tf.concat(
#                 (bi_enc_state[0][0], bi_enc_state[1][0]), -1)
#             encoder_state_h = tf.concat(
#                 (bi_enc_state[0][1], bi_enc_state[1][1]), -1)
#             self.encoder_state = LSTMStateTuple(c = encoder_state_c, h = encoder_state_h)
            
#             encoder_state = []
#             for layer_id in range(num_layers):
#                 encoder_state.append(bi_enc_state[0][layer_id])  # forward
#                 encoder_state.append(bi_enc_state[1][layer_id])  # backward
#             self.encoder_state = tuple(encoder_state)
            
#             self.encoder_state = tf.concat(bi_enc_state, 0)

    def _init_decoder(self):
        def make_cell(rnn_size):
            dec_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
            return dec_cell

        projection_layer = layers_core.Dense(units = self.vocab_size, use_bias=False)
        '''i dont know what this is'''
        self.encoder_state = tuple(self.encoder_state[-1] for _ in range(self.num_layers))
        
        cell = tf.contrib.rnn.MultiRNNCell([make_cell(self.decoder_num_units) for _ in range(self.num_layers)])
        cell_2 = tf.contrib.rnn.MultiRNNCell([make_cell(self.decoder_num_units) for _ in range(self.num_layers)])
        with tf.variable_scope("Decoder") as scope:
            if self.attention:
                # attention_states: [batch_size, max_time, num_units]
                if self.time_major == True:
                    attention_states = tf.transpose(self.encoder_outputs, [1, 0, 2])
                else:
                    attention_states = self.encoder_outputs

                # Create an attention mechanism
                attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                    self.decoder_num_units, 
                    attention_states,
                    memory_sequence_length = self.encoder_inputs_length)

                decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                    cell_2, 
                    attention_mechanism,
                    attention_layer_size = self.decoder_num_units)

                initial_state = decoder_cell.zero_state(batch_size = self.batch_size, dtype = tf.float32)
                initial_state = initial_state.clone(cell_state = self.encoder_state)
            else:
                initial_state = self.encoder_state
            
            # Helper
            training_helper = tf.contrib.seq2seq.TrainingHelper(
                self.decoder_embedding_inputs, 
                self.decoder_train_length, 
                time_major = self.time_major)

            # Decoder
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cell, 
                helper = training_helper, 
                initial_state = initial_state,
                output_layer = projection_layer)

            # Dynamic decoding
            (self.decoder_outputs_train,
            self.decoder_state_train,
            final_sequence_length) = tf.contrib.seq2seq.dynamic_decode(
                    training_decoder, 
                    scope=scope,
                    impute_finished = False
            )
            self.decoder_logits_train = self.decoder_outputs_train.rnn_output
            decoder_predictions_train = tf.argmax(self.decoder_logits_train, axis=-1)
            self.decoder_predictions_train = tf.identity(decoder_predictions_train, name = 'train_predictions')
        
    
        ''' beam search decoder / inference section '''
        # Tile inputs for beam search decoder
        if self.beam_search:
            dec_start_state = seq2seq.tile_batch(self.encoder_state, self.beam_width)
            enc_outputs = seq2seq.tile_batch(self.encoder_outputs, self.beam_width)
            enc_lengths = seq2seq.tile_batch(self.encoder_inputs_length, self.beam_width)
        else:
            dec_start_state = self.encoder_state
            enc_outputs = self.encoder_outputs
            enc_lengths = self.encoder_inputs_length
            
#         with tf.variable_scope("Decoder", reuse = True) as scope:
        scope.reuse_variables()
        if self.attention:
            # attention_states: [batch_size, max_time, num_units]
            if self.time_major == True:
                attention_states = tf.transpose(enc_outputs, [1, 0, 2])
            else:
                attention_states = enc_outputs

            # Create an attention mechanism
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                self.decoder_num_units, 
                attention_states,
                memory_sequence_length = enc_lengths)

            inference_decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell_2,
                attention_mechanism,
                attention_layer_size = self.decoder_num_units)

            if self.beam_search:
                initial_state = inference_decoder_cell.zero_state(self.batch_size * self.beam_width, tf.float32)
            else:
                initial_state = inference_decoder_cell.zero_state(self.batch_size, tf.float32)
            initial_state = initial_state.clone(cell_state = dec_start_state)
        else:
            initial_state = dec_start_state

        start_tokens = tf.tile(tf.constant([0], dtype=tf.int32), [self.batch_size])

        # Decoder
        if self.beam_search == False:
            # Helper
            inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                self.embedding_decoder,
                start_tokens = start_tokens, 
                end_token = 1) # EOS id

            inference_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = inference_decoder_cell, 
                helper = inference_helper, 
                initial_state = initial_state,
                output_layer = projection_layer)
        else:
            inference_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                    cell          = inference_decoder_cell,
                    embedding     = self.embedding_decoder,
                    start_tokens  = tf.ones_like(self.encoder_inputs_length) * tf.constant(0, dtype = tf.int32),
                    end_token     = tf.constant(1, dtype = tf.int32),
                    initial_state = initial_state,
                    beam_width    = self.beam_width,
                    output_layer  = projection_layer)

        # Dynamic decoding
        self.decoder_outputs_inference, __, ___ = tf.contrib.seq2seq.dynamic_decode(
            decoder = inference_decoder,
            maximum_iterations = tf.round(tf.reduce_max(self.encoder_inputs_length)) * 2,
            impute_finished = False)
        
        if self.beam_search:
            self.decoder_predictions_inference = tf.identity(self.decoder_outputs_inference.predicted_ids,
                                                             name = 'inference_predictions')
        else:
            self.decoder_predictions_inference = tf.identity(self.decoder_outputs_inference.sample_id,
                                                             name = 'inference_predictions')


    def _init_optimizer(self):
        
        # Mask out the losses we don't care about
        
        '''
        TOTRY: tf.sequence_loss
        
        '''
        loss_mask = tf.sequence_mask(
            tf.to_int32(self.decoder_targets_length), 
            tf.reduce_max(self.decoder_targets_length),
            dtype = tf.float32)
        
#         crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
#             labels = self.decoder_targets,
#             logits = self.decoder_logits_train
#         )
#         if self.time_major == True:
#             losses = crossent * tf.transpose(tf.to_float(loss_mask), [1, 0])
#         else:
#             losses =  tf.to_float(loss_mask) * crossent

#         train_loss = tf.reduce_sum(losses) / tf.cast(self.batch_size, tf.float32)
#         self.loss = train_loss
        
        self.loss = cost = tf.contrib.seq2seq.sequence_loss(self.decoder_logits_train,
                                                            self.decoder_targets,
                                                            loss_mask)
        tf.summary.scalar('loss', self.loss)
        self.summary_op = tf.summary.merge_all()
        
        learning_rate = 0.0002
        optimizer = tf.train.AdamOptimizer(learning_rate)
        gradients = optimizer.compute_gradients(self.loss)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        self.train_op = optimizer.apply_gradients(capped_gradients)
    
        # Calculate and clip gradients
#         params = tf.trainable_variables()
#         gradients = tf.gradients(train_loss, params)
#         clipped_gradients, _ = tf.clip_by_global_norm(
#             gradients, 
#             5 # max_gradient_norm, usually 5 or 1
#         )
        
#         # Optimization
#         optimizer = tf.train.AdamOptimizer(learning_rate)
#         update_step = optimizer.apply_gradients(
#             zip(clipped_gradients, params))
#         self.train_op = optimizer.minimize(self.loss)

In [73]:
tf.reset_default_graph()
tf.set_random_seed(1)

step = 0
batch_size = 16
max_batches = int(len(X_train_emb) / batch_size)
batches_in_epoch = 40
epoch_be_saved = 10
beam_width = 3

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as session:
    model = Seq2SeqModel(
        encoder_num_units = 512, 
        decoder_num_units = 512, 
        embedding_size = 512,
        num_layers = 4,
        vocab_size = len(vocab_list), 
        batch_size = batch_size,
        bidirectional = False,
        attention = True,
        pre_emb = False,
        time_major = False,
        beam_search = True,
        beam_width = beam_width
    )
    print('model constructed.')
    
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    loss_track = []
    print('start training.')
    summary_writer = tf.summary.FileWriter('../../log', graph_def=sess.graph_def)
    
    for _epoch in range(1, batches_in_epoch + 1):
        for _batch in range(max_batches + 1):
            X_emb, X, y_emb, y = Touka.input_generator(X_train_emb, X_train, y_train_emb, y_train, batch_size)
            feed_dict = model.make_train_inputs(x = X, y = y, e_m = embedding_matrix)

#             writer = tf.train.("../../log/", sess.graph)
            _, l, train_samples, inf_samples, summary_str = session.run([model.train_op, 
                                                            model.loss, 
                                                            model.decoder_predictions_train,
                                                            model.decoder_predictions_inference,
                                                            model.summary_op],
                                                            feed_dict)
            summary_writer.add_summary(summary_str, _epoch * _batch)

#             tf.scalar_summary('loss', l)
#             loss_track.append(l)
#             writer = tf.train.SummaryWriter("../../log/", sess.graph)

            verbose = True
            if verbose:
                if step == 0 or step % 25 == 0:
                    print('step {}'.format(step))
                    print('  minibatch loss: {}'.format(session.run(model.loss, feed_dict)))
                    for i in range(1):
                        train_sentence = ''
                        for word in train_samples[i]:
                            train_sentence += vocab_list[word]
                        print('train logits:')
                        print(train_sentence)
                        
                        for b in range(beam_width):
                            inf_sentence = ''
                            for word in inf_samples[i]:
                                inf_sentence += vocab_list[word[b]]
                            print('inference logits:')
                            print(inf_sentence)
                            print(' ')
                        
#                         inf_sentence = ''
#                         for word in inf_samples[i]:
#                             inf_sentence += vocab_list[word]
#                         print('inference logits:')
#                         print(inf_sentence)
#                         print(' ')
            step += 1
        print(_epoch, 'epoch finished')
        if _epoch % epoch_be_saved == 0:
            saver.save(session, '../../model/' + 'enc_dec_layer_2_dim_512_pre_emb_None_beam_search.ckpt', global_step = step)
            print('model saved at step =', step)
                
    print('finish training')

model constructed.
start training.
step 0
  minibatch loss: 6.51484489440918
train logits:
烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉烤肉
inference logits:
進步進步進步進步進步進步進步進步一定一定一定花花花花花花花花花花花花花花花花人遠離遠離遠離遠離遠離遠離遠離遠離人人人人人人人人人人人人地人人地人人人地人人人人地人人人人人人人人人
 
inference logits:
進步進步進步進步進步進步進步進步一定一定一定花花花花花花花花花花花花花花花花人遠離遠離遠離遠離遠離遠離遠離遠離人人人人人人人人人人人人地人人地人人人地人人人人人人地人人人人人人人
 
inference logits:
進步進步進步進步進步進步進步進步一定一定一定花花花花花花花花花花花花花花花花人遠離遠離遠離遠離遠離遠離遠離遠離人人人人人人人人人人人人地人人地人人人地人人人人地人人人人人人人人地
 
step 25
  minibatch loss: 4.798430442810059
train logits:
GO我我我，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
inference logits:
好好咀嚼咀嚼咀嚼咀嚼記憶記憶記憶失落失落失落失落失落失落失落失落伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出
 
inference logits:
好好咀嚼咀嚼咀嚼咀嚼記憶記憶記憶失落失落失落失落失落失落失落失落失落伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸

12 epoch finished
step 350
  minibatch loss: 4.070202827453613
train logits:
GO我吃飯的在，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，。。。。。。。。。。。。。。
inference logits:
跳樓跳樓咀嚼心裡小孩小孩咀嚼咀嚼咀嚼鬧會長地地只有只有只有只有只有只有變胖變胖變胖變胖變胖變胖變胖變胖變胖變胖版版版版版版版版版版版版變胖變胖變胖變胖變胖變胖推伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出
 
inference logits:
跳樓跳樓咀嚼心裡小孩小孩咀嚼咀嚼咀嚼鬧會長地地只有只有只有只有只有只有變胖變胖變胖變胖變胖變胖變胖變胖變胖版版版版版版版版版版版版版變胖變胖變胖變胖變胖變胖推伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出
 
inference logits:
跳樓跳樓咀嚼心裡小孩小孩咀嚼咀嚼咀嚼鬧會長地地只有只有只有只有只有只有變胖變胖變胖變胖變胖變胖變胖變胖變胖變胖版版版版版版版版版版版版變胖變胖變胖變胖變胖推推伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出
 
step 375
  minibatch loss: 4.0002546310424805
train logits:
GO我大便時在，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，EOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOSEOS
inference logits:
跳樓跳樓心裡心裡小孩小孩咀嚼咀嚼咀嚼鬧鬧地只有只有只有懶惰只有只有只有變胖版版版版版版版版版變胖伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出伸出
 
inference logits:
跳樓跳樓心裡心裡小孩小孩咀嚼咀嚼咀嚼鬧鬧地只

22 epoch finished
step 650
  minibatch loss: 2.0250134468078613
train logits:
GO我unknown的，就去和吃做吃吃屎呼吸，結果讓的，unknown吃就想吃，打得我的東西吃就會邊，吃不邊就會想吃，結果unknown指甲，結果就人，在想。EOS邊睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺
inference logits:
一定咀嚼造成心裡發呆完玩想興奮鬧興奮懶惰懶惰餐廳餐廳餐廳餐廳餐廳摩托車摩托車摩托車摩托車徵兵徵兵徵兵生活悲傷悲傷悲傷悲傷悲傷悲傷銀行銀行銀行銀行銀行銀行銀行銀行銀行悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟
 
inference logits:
一定咀嚼造成心裡發呆完玩想興奮鬧興奮懶惰懶惰餐廳餐廳餐廳餐廳餐廳摩托車摩托車摩托車摩托車徵兵徵兵徵兵生活悲傷悲傷悲傷悲傷悲傷悲傷銀行銀行銀行銀行銀行銀行銀行銀行銀行悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟
 
inference logits:
一定咀嚼造成心裡發呆完玩想興奮鬧興奮懶惰懶惰餐廳餐廳餐廳餐廳餐廳摩托車摩托車摩托車摩托車徵兵徵兵徵兵生活悲傷悲傷悲傷悲傷悲傷悲傷銀行銀行銀行銀行銀行銀行銀行銀行銀行悲傷可愛悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟
 
23 epoch finished
step 675
  minibatch loss: 1.895554542541504
train logits:
GO我常時會家在，變得方便爸爸，我就只好舒服家家，吃一邊，unknown去就講話，便就去，一後去想朋友unknown，便後大便。EOS邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊邊
inference logits:
，咀嚼倒下造成完完玩玩想興奮交到衛生紙刺激衛生紙帥哥懶惰懶惰帥哥帥哥瘋狂摩托車生活正在正在正在正在銀行銀行銀行銀行銀行銀行銀行生病摩托車摩托車摩托車悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷悲傷興奮悲傷興奮悲傷興奮悲傷興奮悲傷興奮悲傷興奮悲

32 epoch finished
step 950
  minibatch loss: 1.2162425518035889
train logits:
GO我壞掉後要吃藥，吃unknown只有喝水，覺得unknownunknown，unknown喝喝覺得放鬆，之後去看放鬆，所以去去，一去就想看韓劇，看韓劇時吵架一，一了睡覺覺得就去unknown，昏倒醫生健康。EOS睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺
inference logits:
，銀行之類之類之類海賊王海賊王海賊王海賊王海賊王海賊王海賊王海賊王搞搞海賊王臉書臉書帥哥帥哥帥哥悲傷臉書悲傷臉書悲傷跑跑跑跑跑跑跑跑跑跑跑跑跑跑悲傷跑一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容
 
inference logits:
，銀行之類之類之類海賊王海賊王海賊王海賊王海賊王海賊王海賊王海賊王搞搞海賊王臉書臉書帥哥帥哥帥哥悲傷臉書悲傷臉書悲傷跑跑跑跑跑跑跑跑跑跑跑跑跑跑悲傷跑一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟一趟笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容
 
inference logits:
，銀行之類之類之類海賊王海賊王海賊王海賊王海賊王海賊王海賊王海賊王搞搞海賊王臉書臉書帥哥帥哥帥哥悲傷臉書悲傷臉書悲傷跑跑跑跑跑跑跑跑跑跑跑跑跑跑悲傷跑一趟一趟一趟一趟一趟一趟一趟一趟一趟笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容笑容
 
33 epoch finished
step 975
  minibatch loss: 0.9476248621940613
train logits:
GO我吃飯unknown伸出unknown拿碗，之後用手去拿了，再去去家事，做家事後覺得unknown，休息牙齒打開，覺得unknownunknown，去去完，覺得完後就去睡覺。EOS昏倒睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺睡覺
inference logits:
，銀行之類之類之類之類強壯強壯強壯睡著睡著刺激搞刺激刺激刺激刺激刺激刺激刺激刺激刺激刺激刺激刺激刺激睡著跑哭泣睡著哭泣睡著帥哥刺激跑一趟一趟一趟

In [ ]:
'''Remember put <GO> in front of the inferenced inputs'''
a = 300
X_test = X_train[a:a+16]
for i in range(0, len(X_test)):
#     X_test[i] = np.concatenate([tokens_embedding[0], X_test[i]])
    X_test[i] = [tokens_id[0]] + X_test[i]
    X_test[i] = X_test[i][0:3]

checkpoint = '../../model/' + 'enc_dec_layer_2_dim_512_pre_emb_None_beam_search.ckpt-' + '1160'

loaded_graph = tf.Graph()
with tf.Session(graph = loaded_graph) as sess:
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    
    '''show trainable variables'''
#     temp_show = [n.name for n in tf.get_default_graph().as_graph_def().node]
#     for o in temp_show:
#         print(o)

    e_i = loaded_graph.get_tensor_by_name('encoder_inputs:0')
#     train_logits = loaded_graph.get_tensor_by_name('Decoder/train_predictions:0')
    inf_logits = loaded_graph.get_tensor_by_name('inference_predictions:0')
    e_i_l = loaded_graph.get_tensor_by_name('encoder_inputs_length:0')
    
    X_test_batch, X_test_batch_length = Touka.prepare_batch(X_test, 
                                                            dim = 2,
#                                                             emb_size = 300, 
                                                            time_major = False)
    
#     answer_logits1 = sess.run(train_logits, {e_i: X_test_batch, e_i_l: X_test_batch_length})
    answer_logits2 = sess.run(inf_logits, {e_i: X_test_batch, e_i_l: X_test_batch_length})

for i in range(batch_size):
    print('===')
#     story_sentence = ''
#     for l in answer_logits1[i]:
#         story_sentence += vocab_list[l]
    
    story_sentence2 = ''
    for l in answer_logits2[i]:
        story_sentence2 += vocab_list[l]
        
    print('concept length by words:', X_test_batch_length[i])
    print(concept[a+i][0:3])
#     print('story length by words:', len(answer_logits1))
#     print(story_sentence)
    print('story length by words:', len(answer_logits2))
    print(story_sentence2)

INFO:tensorflow:Restoring parameters from ../../model/enc_dec_layer_2_dim_512_pre_emb_None_beam_search.ckpt-1160


In [132]:
for o in temp_show:
    print(o)

encoder_inputs
encoder_inputs_length
decoder_targets
decoder_targets_length
DecoderTrainFeeds/Const
DecoderTrainFeeds/Max
DecoderTrainFeeds/ones/shape/0
DecoderTrainFeeds/ones/shape
DecoderTrainFeeds/ones/Const
DecoderTrainFeeds/ones
random_uniform/shape
random_uniform/min
random_uniform/max
random_uniform/RandomUniform
random_uniform/sub
random_uniform/mul
random_uniform
Variable
Variable/Assign
Variable/read
embedding_lookup
random_uniform_1/shape
random_uniform_1/min
random_uniform_1/max
random_uniform_1/RandomUniform
random_uniform_1/sub
random_uniform_1/mul
random_uniform_1
Variable_1
Variable_1/Assign
Variable_1/read
embedding_lookup_1
Encoder/Rank
Encoder/range/start
Encoder/range/delta
Encoder/range
Encoder/concat/values_0
Encoder/concat/axis
Encoder/concat
Encoder/transpose
Encoder/sequence_length
Encoder/rnn/Shape
Encoder/rnn/strided_slice/stack
Encoder/rnn/strided_slice/stack_1
Encoder/rnn/strided_slice/stack_2
Encoder/rnn/strided_slice
Encoder/rnn/strided_slice_1/stack
Enco

gradients/Decoder/Decoder/while/BasicDecoderStep/Decoder/attention_wrapper/attention_wrapper/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/basic_lstm_cell/BiasAdd/Enter_grad/NextIteration
gradients/Decoder/Decoder/while/BasicDecoderStep/Decoder/attention_wrapper/attention_wrapper/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/basic_lstm_cell/BiasAdd/Enter_grad/b_acc_3
gradients/Decoder/Decoder/while/BasicDecoderStep/Decoder/attention_wrapper/attention_wrapper/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/basic_lstm_cell/concat_grad/Rank
gradients/Decoder/Decoder/while/BasicDecoderStep/Decoder/attention_wrapper/attention_wrapper/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/basic_lstm_cell/concat_grad/mod/f_acc
gradients/Decoder/Decoder/while/BasicDecoderStep/Decoder/attention_wrapper/attention_wrapper/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/basic_lstm_cell/concat_grad/mod/RefEnter
gradients/Decoder/Decoder/while/BasicDecoderStep/Decoder/attention_wrapper/attention_wrapper/multi_rnn_cel

gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/BroadcastGradientArgs/StackPush_1
gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/BroadcastGradientArgs/StackPop_1/RefEnter
gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/BroadcastGradientArgs/StackPop_1
gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/BroadcastGradientArgs
gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/mul/f_acc
gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/mul/RefEnter
gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/mul/StackPush
gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/mul/StackPop/RefEnter
gradients_1/Encoder/rnn/while/rnn/multi_rnn_cell/cell_3/cell_3/basic_lstm_cell/mul_2_grad/